# Testing Agentic Planning with ControlFlow

In [24]:
from typing import List, Literal

import controlflow as cf
from dotenv import load_dotenv
from pydantic import BaseModel, Field

load_dotenv()

True

## Multi-Agent Solution
The flow we're eying atm would be this:
```
A: input P -> B [find Ps without NA children]
B -> C{Any found?}
C -> |yes| D[generate children for these Ps]
D -> E[Classify new children as P or NA]
E -> D
C -> |no| F[Check if each P's direct children achieve its outcome]
F -> |Incomplete Ps| D
F -> |all complete| G[Final Task Tree]
```

But let's build towards this gradually.






### V0.0.3 - add Task classification as Project or Next Action

In [25]:
class TaskNode(BaseModel):
    id: str = Field(description="Unique identifier for the task")
    title: str = Field(description="Title/description of the task")
    type: Literal["project", "next_action"] = Field(
        description="Type of the task, either 'project' or 'next_action'"
    )
    subtasks: List["TaskNode"] = Field(default_factory=list)


TaskNode.model_rebuild()


# Result wrapper for ControlFlow
class TaskResult(BaseModel):
    task: TaskNode

In [26]:
def print_task_nodes(task: TaskNode, indent: int = 0):
    """Print task tree recursively with nice formatting.

    Args:
        task: Task to print
        indent: Current indentation level
    """
    # Print current task with indentation
    classification = "P" if task.type == "project" else "NA"
    print("    " * indent + f"{task.id} - {classification} - {task.title}")

    # Recursively print subtasks
    for subtask in task.subtasks:
        print_task_nodes(subtask, indent + 1)

In [29]:
chunker_instructions = """
You are a Getting Things Done (GTD) expert at breaking down projects into subtasks.

Word a subtask in one of two ways:
- either as a concrete, actionable step that can be done in one go without questions
- or as an outcome that needs to be achieved by taking several concrete steps.
"""
chunker = cf.Agent(
    name="Project Chunker",
    model="openai/gpt-4o-mini",
    instructions=chunker_instructions,
)

classifier_instructions = """
You are a Getting Things Done (GTD) expert at analyzing and classifying tasks.

IMPORTANT: For each task, you MUST include:
1. id: String (use format: parent_id + letter, e.g., "1.a", "1.b", "1.c")
2. title: String (the task description)
3. type: Either "project" or "next_action"

CRITICAL CHECK: Before classifying any task as NA, ask yourself:
1. Can this be done in a single sitting without interruption?
2. Do I know EXACTLY what physical action to take first?
3. Is there ANY planning or decision-making needed before starting?
4. Does this involve multiple tools, steps, or sessions?

AUTOMATIC PROJECT TRIGGERS - Mark as P if the task:
1. Contains words like:
   - "Update", "Prepare", "Research", "Network"
   - "Get ready", "Set up", "Develop", "Build"
2. Requires ongoing effort or multiple sessions
3. Needs decisions about "how" or "what" before starting
4. Could be broken down into multiple smaller tasks

Examples with analysis:

"Update your resume" -> PROJECT
Why? 
- Requires multiple decisions (what to add/remove)
- Multiple sections to work on
- Needs review and iteration
Better wording: "Create targeted resume for genAI positions (P)"

"Research companies" -> PROJECT
Why?
- Where to research?
- What criteria to use?
- How to track findings?
Better wording: "Create shortlist of potential employer companies (P)"

"Network with professionals" -> PROJECT
Why?
- Who to contact?
- What to say?
- Multiple interactions needed
Better wording: "Build professional network in genAI field (P)"

"Prepare for interviews" -> PROJECT
Why?
- Multiple topics to cover
- Requires practice sessions
- Needs materials and planning
Better wording: "Develop interview preparation system (P)"

TRUE NEXT ACTIONS examples:
- "Send connection request to Sarah Smith with drafted message"
- "Add Python certification to resume's Skills section"
- "Schedule mock interview with John for Tuesday at 2pm"

Remember: If you can't immediately start the task RIGHT NOW with NO planning,
it's a PROJECT, not a Next Action!
"""

classifier = cf.Agent(
    name="Task Classifier",
    model="openai/gpt-4o-mini",
    instructions=classifier_instructions,
)


@cf.flow
def chunk_flow_v0_0_3(project: str):
    # Initialize with root project
    root_task_id = "1"
    root_task = TaskNode(id=root_task_id, title=project, type="project")

    with cf.Task(
        "Generate hyrarchical task tree for this project, where each task is a \
            either a project or a next action",
        result_type=TaskResult,
        agents=[chunker, classifier],
    ) as main_task:
        task_tree = cf.run(
            """
            Break given task into subtasks.
            Return a TaskResult containing a TaskNode with:
            - The root task
            - 3-5 subtasks in the subtasks list
            Once you split the task into subtasks, mark this task as complete.
            """,
            context=dict(
                task=root_task,
            ),
            agents=[chunker],
            result_type=TaskResult,
            tools=[main_task.get_success_tool()],
        )

        classified_tree = cf.run(
            """
            Classify each subtask as either 'project' or 'next_action'.
            Return the complete TaskResult with classifications added.
            Once done, use the main task tool to mark the entire process as complete.
            """,
            context=dict(task_tree=task_tree),
            agents=[classifier],
            result_type=TaskResult,
            tools=[main_task.get_success_tool()],
        )

        # tasks = TaskList(tasks=tasks.tasks + new_tasks.tasks)

    return classified_tree.task


tasks = chunk_flow_v0_0_3("Get a job as a genAI engineer")

print(tasks)

Output()

Output()

id='1' title='Get a job as a genAI engineer' type='project' subtasks=[TaskNode(id='1.1', title='Update resume to highlight relevant skills and experiences for genAI engineering jobs', type='next_action', subtasks=[]), TaskNode(id='1.2', title='Create tailored cover letters for at least three specific job postings', type='next_action', subtasks=[]), TaskNode(id='1.3', title='Research companies hiring genAI engineers and make a list of potential employers', type='project', subtasks=[]), TaskNode(id='1.4', title='Network with professionals in the genAI field through LinkedIn or industry events', type='project', subtasks=[]), TaskNode(id='1.5', title='Prepare for technical interviews by practicing coding challenges and genAI concepts', type='project', subtasks=[])]


In [30]:
print_task_nodes(tasks)

1 - P - Get a job as a genAI engineer
    1.1 - NA - Update resume to highlight relevant skills and experiences for genAI engineering jobs
    1.2 - NA - Create tailored cover letters for at least three specific job postings
    1.3 - P - Research companies hiring genAI engineers and make a list of potential employers
    1.4 - P - Network with professionals in the genAI field through LinkedIn or industry events
    1.5 - P - Prepare for technical interviews by practicing coding challenges and genAI concepts
